<a href="https://colab.research.google.com/github/aaronzr/langchain-practice/blob/main/docs/docs/how_to/self_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install --quiet lark langchain_ollama langchain_community langchain_core pypdf langchain_text_splitters langchain_chroma

OLLAMA_BASE_URL = "https://2d8b-134-79-78-85.ngrok-free.app"

MODEL_NAME = "llama3.1:latest"
# MODEL_NAME = "dolphin3:latest"
# MODEL_NAME = "deepseek-r1:8b"

In [26]:
# test ollama connection

import requests

url = f"{OLLAMA_BASE_URL}/api/generate"
payload = {
    "model": MODEL_NAME,
    "message": "hi"
}

try:
    response = requests.post(url, json=payload)
    response.raise_for_status()  # Raise an exception for bad status codes
    print("Connection successful!")
    print(response.json())  # Print the response from Ollama
except requests.exceptions.RequestException as e:
    print(f"Connection error: {e}")

Connection successful!
{'model': 'llama3.1:latest', 'created_at': '2025-03-28T19:27:05.18252636Z', 'response': '', 'done': True, 'done_reason': 'load'}


# How to do "self-querying" retrieval

:::info

Head to [Integrations](/docs/integrations/retrievers/self_query) for documentation on vector stores with built-in support for self-querying.

:::

A self-querying [retriever](/docs/concepts/retrievers/) is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to its underlying [vector store](/docs/concepts/vectorstores/). This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documents but to also extract filters from the user query on the metadata of stored documents and to execute those filters.

![](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/self_querying.jpg?raw=1)

## Get started
For demonstration purposes we'll use a `Chroma` vector store. We've created a small demo set of documents that contain summaries of movies.

**Note:** The self-query retriever requires you to have `lark` package installed.

In [27]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, OllamaEmbeddings(base_url = OLLAMA_BASE_URL, model = "mxbai-embed-large", temperature = 0))

### Creating our self-querying retriever

Now we can instantiate our retriever. To do this we'll need to provide some information upfront about the metadata fields that our documents support and a short description of the document contents.

In [30]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_ollama import ChatOllama

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = ChatOllama(model="llama3.1", base_url=OLLAMA_BASE_URL, temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

### Testing it out

And now we can actually try using our retriever!

In [31]:
# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(id='68c8d553-9cce-4ed9-b33b-e9cf496f1406', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(id='c7712fb8-8613-4427-8f4c-1513aac3654a', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}, page_content='Three men walk into the Zone, three men walk out of the Zone')]

In [32]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

[]

In [33]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

OutputParserException: Parsing text
Here is the structured request schema in JSON format:

```json
{
    "query": "highly rated science fiction",
    "filter": "gt(\"rating\", 8.5) and eq(\"genre\", \"science fiction\")"
}
```

Explanation:

* The query string contains the text that is expected to match the contents of documents, which are the keywords "highly rated" and "science fiction".
* The filter statement uses the `gt` comparator to compare the value of the "rating" attribute with 8.5, and the `eq` comparator to check if the value of the "genre" attribute is equal to "science fiction".
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 19.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [34]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[Document(id='569afedf-fb75-45a5-ac8b-c1369a15685c', metadata={'genre': 'animated', 'year': 1995}, page_content='Toys come alive and have a blast doing so')]

### Filter k

We can also use the self query retriever to specify `k`: the number of documents to fetch.

We can do this by passing `enable_limit=True` to the constructor.

In [35]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
)

# This example only specifies a relevant query
retriever.invoke("What are two movies about dinosaurs")

[Document(id='4b858b43-ef85-4e63-bdb0-60adec23237e', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(id='569afedf-fb75-45a5-ac8b-c1369a15685c', metadata={'genre': 'animated', 'year': 1995}, page_content='Toys come alive and have a blast doing so')]

## Constructing from scratch with LCEL

To see what's going on under the hood, and to have more custom control, we can reconstruct our retriever from scratch.

First, we need to create a query-construction chain. This chain will take a user query and generated a `StructuredQuery` object which captures the filters specified by the user. We provide some helper functions for creating a prompt and output parser. These have a number of tunable params that we'll ignore here for simplicity.

In [36]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

Let's look at our prompt:

In [37]:
print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

And what our full chain produces:

In [38]:
query_constructor.invoke(
    {
        "query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"
    }
)

StructuredQuery(query='taxi driver', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction'), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=1990), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Luc Besson')]), limit=None)

The query constructor is the key element of the self-query retriever. To make a great retrieval system you'll need to make sure your query constructor works well. Often this requires adjusting the prompt, the examples in the prompt, the attribute descriptions, etc. For an example that walks through refining a query constructor on some hotel inventory data, [check out this cookbook](https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb).

The next key element is the structured query translator. This is the object responsible for translating the generic `StructuredQuery` object into a metadata filter in the syntax of the vector store you're using. LangChain comes with a number of built-in translators. To see them all head to the [Integrations section](/docs/integrations/retrievers/self_query).

In [39]:
from langchain_community.query_constructors.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),
)

In [40]:
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[Document(id='569afedf-fb75-45a5-ac8b-c1369a15685c', metadata={'genre': 'animated', 'year': 1995}, page_content='Toys come alive and have a blast doing so')]